In [24]:
# Author: Davide Aloi - PhD student - University of Birmingham
# Description: the script loads the results from wp_all_8_current_density_metrics, which are 3
# .csv files, and correlates current density metrics with DCM values.
# The analysis method used is Canonical correlation analysis (CCA)
# Add details on CCA
## WORKING ON THIS AT THE MOMENT (25012022)
# Imports
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd
import scipy.stats

In [25]:
## Parameters and variables: 
results_folder = 'D:\\roast-chapter3\\wp_all_results\\' # Folder with results (current density)
main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\' # Project folder
dcm_results_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\all_dcm_results\\' # Folder with dcm results

# Datasets names and subjects lists
db_names = ['wp2a', 'wp1b'] # waiting for wp1a results from Davinia

In [26]:
for db_id, db in enumerate(db_names):

    # Loading DCM results (nb: I load the unthresholded ones, for the correlation analysis)
    DCM = np.load(os.path.join(dcm_results_folder, db + '_dcm_unthresholded.npy'))
    DCM_sham = np.load(os.path.join(dcm_results_folder, db + '_dcm_sham_unthresholded.npy'))
    # Loading current density metrics
    cd_metrics = pd.read_csv(os.path.join(results_folder, db +'_current_density_results.csv'))

    # Simple R correlation (example)
    x = cd_metrics['m1_medians'].values # Medians current density in M1
    y = DCM[:, 0, 0] # NB: DCM index related to m1 <-> m1 (of all subjects) = [0,0]
    r,p  = scipy.stats.pearsonr(x, y)    # Pearson's r
    print ('R: ' + str(round(r,3)) + ', p: ' +  str(round(p,3)) )

R: -0.029, p: 0.899
R: -0.192, p: 0.404


In [ ]:
for db_id, db in enumerate(db_names):
    